In [190]:
import pandas as pd
import os
import sys
import json
from datasets import load_dataset
import random
import openai
from openai import OpenAI

sys.path.insert(0, '/home/al2644/research/')
import openaiAPI
from tqdm import tqdm

In [245]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
from dateutil.parser import parse
from dateutil.parser._parser import ParserError

In [237]:
url = 'https://en.wikipedia.org/wiki/New%20York%20Yankees'# 'https://en.wikipedia.org/wiki/New%20York%20Yankees'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) ' 
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/129.0.0.0 Safari/537.36'
}
response = requests.get(url, headers = headers, timeout = 10)

In [238]:
response.raise_for_status()

In [239]:
page_content = response.content
soup = BeautifulSoup(page_content, 'html.parser')

In [240]:
ref_cols = soup.select('div.reflist')

In [256]:
li = ref_cols[1].find('ol', class_='references').find_all('li', id = True)[0]
pattern = r'(.*)\b(\d{4})\b'
text = li.get_text().split('Retrieved')[1]

import re
match = re.search(pattern, text)
match.group(2)

'2022'

In [253]:
text

'^ a b c d Hoch, Bryan (February 4, 2021). "NYPD & Tiffany: The story behind Yanks\' logo". MLB Advanced Media. Retrieved October 21, 2022. The interlocking "NY" of the Yankees\' logo is arguably the most recognizable in all of professional sports, spotted on streets from The Bronx to Beijing, Manhattan to Melbourne. Their navy blue and white caps have transcended baseball, becoming a global cultural touchstone.\n'

In [40]:
li_elements = references.find_all('li', id=True)

In [170]:
reference_text = li_elements[0].find('span', class_='reference-text')

In [173]:
a_tags = reference_text.find_all('a', href = True)

# Extract Links

In [260]:
from codebase.wiki_entities_knowledge.wiki_scrapper import scrape_reference
import importlib; importlib.reload(scrape_reference)

root = '/share/goyal/lio/knowledge_update/wikipedia/20231101.en/'

In [225]:
step3_cache_df = pd.read_csv(os.path.join(root, 'select_sample_step3_output.csv')).set_index('index')

In [262]:
wiki_reference_df = pd.read_pickle(os.path.join(root, 'select_sample_wiki_references.pickle'))

In [263]:
wiki_reference_df

,url,title,wiki_reference
index,,,
8,https://en.wikipedia.org/wiki/British%20Museum,British Museum,https://www.theguardian.com/artanddesign/2022/...
20,https://en.wikipedia.org/wiki/New%20York%20Yan...,New York Yankees,https://www.nydailynews.com/sports/baseball/go...
29,https://en.wikipedia.org/wiki/Cayan%20Tower,Cayan Tower,https://www.emirates247.com/news/emirates/beyo...
33,https://en.wikipedia.org/wiki/Brenner%20Base%2...,Brenner Base Tunnel,https://doi.org/10.1007%2Fs00703-008-0327-2 ...
43,https://en.wikipedia.org/wiki/Oktoberfest,Oktoberfest,https://oktoberfest2024.us/hofbrau-festzelt-te...
53,https://en.wikipedia.org/wiki/Cornell%20Univer...,Cornell University,https://www.bloomberg.com/bw/articles/2013-01-...
70,https://en.wikipedia.org/wiki/Bill%20Ackman,Bill Ackman,https://www.theguardian.com/us-news/2017/apr/1...
78,https://en.wikipedia.org/wiki/Fast%20%26%20Fur...,Fast & Furious,https://deadline.com/2019/06/john-cena-officia...
90,https://en.wikipedia.org/wiki/Bitcoin,Bitcoin,https://techcrunch.com/2013/08/09/bitcoin-tick...


In [264]:
'''grab dates'''
dates = []
for i in tqdm(range(len(wiki_reference_df))):
    url = wiki_reference_df['url'].iloc[i]
    link_date_dict = scrape_reference.get_reference_retrieved_date(url)
    dates.append(link_date_dict)
wiki_reference_df['wiki_reference_date'] = dates

100%|█████████████████████████████████████████| 35/35 [00:24<00:00,  1.43it/s]


In [266]:
'''merge date and content'''
link_content_dates = []
for i in tqdm(range(len(wiki_reference_df))):
    wiki_reference = wiki_reference_df['wiki_reference'].iloc[i].reset_index().rename(columns = {'index': 'link', 0: 'content'})
    wiki_reference_date = pd.DataFrame.from_dict(wiki_reference_df['wiki_reference_date'].iloc[i], orient='index').reset_index().rename(columns = {'index': 'link', 0: 'date'})
    reference_content_date = wiki_reference.merge(wiki_reference_date, on = 'link')
    link_content_dates.append(reference_content_date)

100%|████████████████████████████████████████| 35/35 [00:00<00:00, 249.11it/s]


In [270]:
wiki_reference_df['noisy_articles'] = link_content_dates

In [274]:
output_df = wiki_reference_df.drop(columns = ['wiki_reference', 'wiki_reference_date'])

In [276]:
filename = 'select_sample_wiki_references_with_date.pickle'
output_df.to_pickle(os.path.join(root, filename))

# Extract link dates

In [178]:
url = 'https://en.wikipedia.org/wiki/British%20Museum'
link_dates = scrape_reference.get_reference_retrieved_date(url)

> /home/al2644/research/codebase/wiki_entities_knowledge/wiki_scrapper/scrape_reference.py(47)get_reference_retrieved_date()
     45 def get_reference_retrieved_date (wiki_url):
     46     import pdb; pdb.set_trace()
---> 47     response = requests.get(wiki_url, headers=headers, timeout=10)
     48     response.raise_for_status()
     49 

ipdb> n
> /home/al2644/research/codebase/wiki_entities_knowledge/wiki_scrapper/scrape_reference.py(48)get_reference_retrieved_date()
     46     import pdb; pdb.set_trace()
     47     response = requests.get(wiki_url, headers=headers, timeout=10)
---> 48     response.raise_for_status()
     49 
     50     links_dates = {}

ipdb> n
> /home/al2644/research/codebase/wiki_entities_knowledge/wiki_scrapper/scrape_reference.py(50)get_reference_retrieved_date()
     48     response.raise_for_status()
     49 
---> 50     links_dates = {}
     51 
     52     soup = BeautifulSoup(response.content, 'html.parser')

ipdb> n
> /home/al2644/research/codebase/wi

ipdb> n
> /home/al2644/research/codebase/wiki_entities_knowledge/wiki_scrapper/scrape_reference.py(63)get_reference_retrieved_date()
     61 
     62         for li in li_elements:
---> 63             '''get date'''
     64             text = li.get_text()
     65             pattern = r'Retrieved\s+(\d{1,2}\s+(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4})'

ipdb> n
> /home/al2644/research/codebase/wiki_entities_knowledge/wiki_scrapper/scrape_reference.py(64)get_reference_retrieved_date()
     62         for li in li_elements:
     63             '''get date'''
---> 64             text = li.get_text()
     65             pattern = r'Retrieved\s+(\d{1,2}\s+(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4})'
     66             match = re.search(pattern, text, re.IGNORECASE)

ipdb> n
> /home/al2644/research/codebase/wiki_entities_knowledge/wiki_scrapper/scrape_reference.py(65)get_reference_ret

ipdb> date
'22 July 2016'
ipdb> n
> /home/al2644/research/codebase/wiki_entities_knowledge/wiki_scrapper/scrape_reference.py(74)get_reference_retrieved_date()
     72 
     73             reference_text = li.find('span', class_='reference-text')
---> 74             if reference_text:
     75                 a_tags = reference_text.find_all('a', href = True)
     76                 for a_tag in a_tags:

ipdb> n
> /home/al2644/research/codebase/wiki_entities_knowledge/wiki_scrapper/scrape_reference.py(75)get_reference_retrieved_date()
     73             reference_text = li.find('span', class_='reference-text')
     74             if reference_text:
---> 75                 a_tags = reference_text.find_all('a', href = True)
     76                 for a_tag in a_tags:
     77                     href = a_tag['href']

ipdb> n
> /home/al2644/research/codebase/wiki_entities_knowledge/wiki_scrapper/scrape_reference.py(76)get_reference_retrieved_date()
     74             if reference_text:
  

ipdb> c


In [179]:
link_dates

{'https://www.britishmuseum.org/about_us/management/about_us.aspx': '22 July 2016',
 'https://www.bbc.co.uk/news/uk-england-london-68577122': '18 March 2024',
 'https://www.forbes.com/sites/rsmdiscovery/2017/10/30/ranking-the-worlds-most-admired-art-museums-and-what-big-business-can-learn-from-them/?sh=1f0cee263b33': '18 May 2023',
 'http://www.britishmuseum.org/about_us/the_museums_story/general_history.aspx': '12 July 2018',
 'https://www.bl.uk/events/the-life-and-curiosity-of-hans-sloane': '21 October 2017',
 'https://www.independent.co.uk/news/uk/this-britain/the-big-question-what-is-the-rosetta-stone-and-should-britain-return-it-to-egypt-1836610.html': '2 April 2020',
 'https://www.theguardian.com/culture/2015/jun/29/museums-looting-art-artefacts-world-culture': '18 April 2018',
 'https://www.legislation.gov.uk/ukpga/1972/54/contents': '22 July 2022',
 'https://www.britishmuseum.org/visiting/admission_and_opening_times.aspx': '4 July 2010',
 'https://www.bbc.co.uk/history/british/

# O-1 code

In [ ]:
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time

# List of Wikipedia URLs
wikipedia_urls = [
    'https://en.wikipedia.org/wiki/Python_(programming_language)',
    # Add more URLs as needed
]

# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
}

for wiki_url in wikipedia_urls:
    try:
        # Check if URL is accessible
        response = requests.get(wiki_url, headers=headers, timeout=10)
        if response.status_code == 200:
            print(f"\nAccessing {wiki_url}")
            page_content = response.content
            soup = BeautifulSoup(page_content, 'html.parser')

            # Find the references section
            references = soup.find('ol', class_='references')
            if references:
                # Collect all links in the references section
                reference_links = set()
                for ref in references.find_all('a', href=True):
                    href = ref['href']
                    # Resolve relative URLs
                    full_url = urljoin(wiki_url, href)
                    parsed_url = urlparse(full_url)
                    # Filter out Wikipedia internal links
                    if 'wikipedia.org' not in parsed_url.netloc:
                        reference_links.add(full_url)

                print(f"Found {len(reference_links)} external reference links.")

                # Scrape content from each reference link
                for ref_url in reference_links:
                    try:
                        ref_response = requests.get(ref_url, headers=headers, timeout=10)
                        if ref_response.status_code == 200:
                            ref_content = ref_response.content
                            # Process the content as needed
                            print(f"Scraped content from {ref_url}")
                            # Optional: Sleep to respect rate limiting
                            time.sleep(1)
                        else:
                            print(f"Failed to access {ref_url}: Status code {ref_response.status_code}")
                    except requests.RequestException as e:
                        print(f"Error accessing {ref_url}: {e}")
            else:
                print(f"No references found in {wiki_url}")
        else:
            print(f"Failed to access {wiki_url}: Status code {response.status_code}")
    except requests.RequestException as e:
        print(f"Error accessing {wiki_url}: {e}")
